In [ ]:
import time
import json
import httpx


URL = "http://localhost:8000/chat"

def to_message(role: str, content: str, name: str = None) -> dict:
    message = {
        "role": role,
        "content": content  
    }
    if name is not None:
        message["name"] = name
    return message

def to_user_message(text: str, name: str = None) -> dict: 
    return to_message("user", text, name)

def to_assistant_message(text: str, name: str = None) -> dict:
    return to_message("assistant", text, name)

def test_api(data):
    start_time = time.time()
    end_time = None
    with httpx.stream("POST", URL, json=data) as response:
        for chunk in response.iter_lines():
            json_chunk = json.loads(chunk)
            content = json_chunk["content"]

            if json_chunk["status"] == "progress":
                if end_time is None:
                    end_time = time.time()
                    print("스트리밍 시작:")

                print(content, end="", flush=True)

            elif json_chunk["status"] == "done":
                report = content["report"]
                completion = content["completion"]
                
                print(f"\n\nAnswer: {completion['answer']}")
                for follow_up in completion['follow_up_questions']:
                    print(f"        - {follow_up}")
                print("\nReport:")
                print(f"    - 사용 모델: {report['model']}")
                print(f"    - 입력 토큰: {report['prompt_tokens']}")
                print(f"    - 캐시 히트: {report['cache_hit_tokens']}")
                print(f"    - 출력 토큰: {report['completion_tokens']}")
                print(f"    - 첫 토큰 지연: {end_time - start_time}")

ModuleNotFoundError: No module named 'main'

In [6]:
example_data1 = {
  "conversation_history": [],
  "user_query": "배송 취소하고 싶어요"
}

test_api(example_data1)

스트리밍 시작:
{"answer":"배송 취소는 주문 상태에 따라 처리 방법이 다릅니다. 아직 '집하' 되지 않은 경우, [판매관리 > 발주(주문)확인/발송관리]에서 해당 주문을 선택 후 '집하취소'를 하시고 취소를 진행하시면 됩니다. 이미 발송되기 전이라면, [판매관리 > 취소관리]에서 주문을 선택해 '취소 완료처리' 또는 '취소 거부처리'를 해주세요. 취소는 4영업일 내에 처리하지 않으면 자동 환불 처리됩니다.","follow_up_questions":["발송 후 취소 요청 시 어떻게 해야 하나요?","부분 취소는 가능한가요?"]}

Answer: 배송 취소는 주문 상태에 따라 처리 방법이 다릅니다. 아직 '집하' 되지 않은 경우, [판매관리 > 발주(주문)확인/발송관리]에서 해당 주문을 선택 후 '집하취소'를 하시고 취소를 진행하시면 됩니다. 이미 발송되기 전이라면, [판매관리 > 취소관리]에서 주문을 선택해 '취소 완료처리' 또는 '취소 거부처리'를 해주세요. 취소는 4영업일 내에 처리하지 않으면 자동 환불 처리됩니다.
        - 발송 후 취소 요청 시 어떻게 해야 하나요?
        - 부분 취소는 가능한가요?

Report:
    - 사용 모델: gpt-4.1-mini
    - 입력 토큰: 2152
    - 캐시 히트: 1536
    - 출력 토큰: 169
    - 첫 토큰 지연: 3.0562729835510254


In [7]:
example_data2 = {
  "conversation_history": [
    to_user_message("스토어 교육"),
    to_assistant_message("스마트스토어 운영 시 도움이 될 만한 강의를 온라인으로 확인하실 수 있습니다. 네이버 비즈니스 스쿨에서 교육을 수강 신청 후, 다양한 콘텐츠를 보실 수 있습니다."),
  ],
  "user_query": "산타에게 크리스마스 선물을 어떤 걸 달라고 할까?"
}

test_api(example_data2)

스트리밍 시작:
{"answer":"저는 네이버 스마트스토어 관련 질문에 도움을 드리는 챗봇입니다. 스마트스토어에 관한 궁금한 점을 물어봐 주세요.","follow_up_questions":["스마트스토어에서 크리스마스 이벤트 진행 방법이 궁금하신가요?","스마트스토어 상품 등록에 대해 안내해 드릴까요?"]}

Answer: 저는 네이버 스마트스토어 관련 질문에 도움을 드리는 챗봇입니다. 스마트스토어에 관한 궁금한 점을 물어봐 주세요.
        - 스마트스토어에서 크리스마스 이벤트 진행 방법이 궁금하신가요?
        - 스마트스토어 상품 등록에 대해 안내해 드릴까요?

Report:
    - 사용 모델: gpt-4.1-mini
    - 입력 토큰: 1044
    - 캐시 히트: 0
    - 출력 토큰: 93
    - 첫 토큰 지연: 1.9191431999206543
